## vector stores and retrievers
This topic explains about retrieving data from vector databases for integration with LLM workflows

We will cover
* Documents
* Vector stores
* Retrievers

### Documents
Langchain implements a Document abstraction, which is intendded to represent a unit of text and associated metadata. It has two attributes

* page_content: a string representing the content
* metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about thesource of the document, its relationship to other documents, and other information. Note that an individual Document  object often represents a chunk of a larger document

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [5]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002944A729B80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002944A72A690>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [8]:
## VectorStores
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [9]:
vectorstore.similarity_search("cat")

[Document(id='787198aa-f727-4006-831c-d4875f129b01', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e62f5665-a59c-4fa4-a0d9-9dea95abe64c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bf20d719-776f-4e7a-99bb-8b270325f296', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='ce698c37-896e-4217-9535-7f363991c8bd', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [10]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='787198aa-f727-4006-831c-d4875f129b01', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e62f5665-a59c-4fa4-a0d9-9dea95abe64c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bf20d719-776f-4e7a-99bb-8b270325f296', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='ce698c37-896e-4217-9535-7f363991c8bd', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [11]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='787198aa-f727-4006-831c-d4875f129b01', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351054430007935),
 (Document(id='e62f5665-a59c-4fa4-a0d9-9dea95abe64c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='bf20d719-776f-4e7a-99bb-8b270325f296', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.595690369606018),
 (Document(id='ce698c37-896e-4217-9535-7f363991c8bd', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

### Retrievers

Langchain vectorstore object do not subclass Runnable, and so cannot immediately be integrated into Langchain Expression Langchainn chains

Langchain Retrievers are Runnables.  so they implemented a standard set of methods, (synchronous and asynchronous invoke are batch operations) designed to incorporated in LCEL chains

In [12]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='787198aa-f727-4006-831c-d4875f129b01', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e62f5665-a59c-4fa4-a0d9-9dea95abe64c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [13]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat", "dog"])

[[Document(id='787198aa-f727-4006-831c-d4875f129b01', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e62f5665-a59c-4fa4-a0d9-9dea95abe64c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """ 
Answer this question using the provided context only

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain={"context":retriever, "question": RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.


In [ ]:
from langchain_chroma import Chroma
from langchain_community.prompts import ChatPromptTemplate